# Thai Sentence Segmentation with mT5

## Setup

In [ ]:
import pandas as pd
import glob
import sys
import os

## Import Data

In [ ]:
# For LST20 Corpus, we need to:
# 1. Manually download `AIFORTHAI-LST20Corpus.tar.gz` from https://aiforthai.in.th/corpus.php (login required)
# 2. Extract the .tar.gz; this will result in folder `LST20Corpus`
# In this case, we put the 'train', 'dev', and 'tests' folders in our Google Drive

# For our annotation corpus, we split the file into train, dev, and test.
# We also put the files in the same LST20 folders for convenience.

In [ ]:
from google.colab import  drive
drive.mount('/content/drive')

In [ ]:
def get_df_limittoken(filepath,num_token=150):
    keyList = ['tokens', 'boundary']
    data = {key: [] for key in keyList}
    for filename in glob.glob(os.path.join(filepath, '*.txt')):
        with open(os.path.join(os.getcwd(), filename), 'r') as f:
            column1 = []
            column2 = []
            count_token = 0
            for i,line in enumerate(f.readlines()):
                entry = line.rstrip('\n').split('\t')
                if(len(entry)!=4):  # ถ้าเป็นวรรคแบ่งประโยค
                    entry = [' ','space','space','space']
                if(count_token==num_token):
                    data["tokens"].append(column1)
                    data["boundary"].append(column2)
                    column1 = []
                    column2 = []
                    count_token = 0
                column1.append(str(entry[0]).replace('_',' '))
                #replace _ with space
                if(entry[3]=='space'):
                    # column5.append('SEP')
                    column2.append('|')
                else:               
                    # column5.append('N')
                    column2.append(str(entry[0]).replace('_',' '))
                count_token+=1
            data["tokens"].append(column1)
            data["boundary"].append(column2)
    df = pd.DataFrame.from_dict(data)
    return df

In [ ]:
train_df = get_df_limittoken('/content/drive/My Drive/IS/LST_20/train/')
dev_df = get_df_limittoken('/content/drive/My Drive/IS/LST_20/dev/')
test_df = get_df_limittoken('/content/drive/My Drive/IS/LST_20/test/')

In [ ]:
def source(text):
  new_text = ''.join(text)
  prompt = "segment sentence: "
  # prompt = "segment each sentence with |: "
  # prompt = "ตัดประโยค: "
  # prompt = "ตัดประโยคด้วยเครื่องหมาย |: "
  new_text = prompt + new_text
  return new_text

def target(text):
  return ''.join(text)

def convert_text(df):
  df['source_text'] = df['tokens'].apply(source)
  df['target_text'] = df['boundary'].apply(target)

  return df[['source_text', 'target_text']]

In [ ]:
train_df = convert_text(train_df)
train_df

In [ ]:
dev_df = convert_text(dev_df)
dev_df

In [ ]:
test_df = convert_text(test_df)
test_df

# T5

In [ ]:
# Check whether Colab is connected to GPU.  
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
!pip install --upgrade simplet5

In [ ]:
# import
from simplet5 import SimpleT5

# instantiate
model = SimpleT5()

# load (supports t5, mt5, byT5 models)
# model.from_pretrained("mt5","google/mt5-small")
model.from_pretrained("mt5","google/mt5-base")

In [ ]:
# train
model.train(train_df = train_df, # pandas dataframe with 2 columns: source_text & target_text
            eval_df = test_df, # pandas dataframe with 2 columns: source_text & target_text
            source_max_token_len = 150, 
            target_max_token_len = 150,
            batch_size = 8,
            max_epochs = 3,
            use_gpu = True,            
            outputdir = "outputs",
            early_stopping_patience_epochs = 0,
            precision = 32
            )

# Prediction

In [ ]:
# load trained T5 model

# trained_model = the location of trained mT5 model with the lowest loss
# For example, the location is '/content/content/outputs/simplet5-epoch-0-train-loss-0.606-val-loss-0.1852'
model.load_model("mt5", trained_model, use_gpu=True)

In [ ]:
text = 'segment sentence: ตั้งใจของตนเอง ส่วนผู้ที่จะทำโผแต่งตั้งโยกย้าย ต้องเป็นไปตามกฎหมาย อัศวินข้องใจ"วรวุฒิ"ใช้รถของกลางดีเอสไอ พล.ต.ท.อัศวิน ขวัญเมือง ผู้ช่วย ผบ.ตร. หนึ่งในชุดคลี่คลายคดีลอบยิงนายสนธิ เปิดเผยว่า เมื่อช่วงเช้าที่ผ่านมา ได้ส่งกำลังไปค้นบ้านแม่ยายของ ส.ต.ท.วรวุฒิมุ่งสันติ ตำรวจสังกัดกองบัญชาการตำรวจปราบปรามยาเสพติด (บช.ปส.) ย่านลาดปลาเค้า พบรถยนต์ยี่ห้อเชฟโรเลต รุ่นซาฟีรา สีบรอนซ์ หมายเลขทะเบียน ศว 8051 กรุงเทพมหานคร ซึ่งเป็นทะเบียนปลอม โดยรถคันนี้มีหมายเลขทะเบียนที่แท้จริงคือ กษ 3737 เชียงใหม่ ผู้ครอบครองชื่อ นายชาญณรงค์ มูเซอ ผู้ต้องหาคดีค้าเฮโรอีน ซึ่งกรมสอบสวนคดีพิเศษ'
model.predict(text)

['ตั้งใจของตนเอง ส่วนผู้ที่จะทําโผแต่งตั้งโยกย้าย ต้องเป็นไปตามกฎหมาย|อัศวินข้องใจ"วรวุฒิ"ใช้รถของกลางดีเอสไอ|พล.ต.ท.อัศวิน ขวัญเมือง ผู้ช่วย ผบ.ตร. หนึ่งในชุดคลี่คลายคดีลอบยิงนายสนธิ เปิดเผยว่า เมื่อช่วงเช้าที่ผ่านมา ได้ส่งกําลังไปค้นบ้านแม่ยายของ ส.ต.ท.วรวุฒิมุ่งสันติ ตํารวจสังกัดกองบัญชาการตํารวจปราบปรามยาเสพติด (บช.ปส.) ย่านลาดปลาเค้า พบรถยนต์ยี่ห้อเชฟโรเลต รุ่นซาฟีรา สีบรอนซ์ หมายเลขทะเบียน ศว 8051 กรุงเทพมหานคร ซึ่งเป็นทะเบียนปลอม โดยรถคันนี้มีหมายเลขทะเบียนที่แท้จริงคือ กษ 3737 เชียงใหม่ ผู้ครอบครองชื่อ นายชาญณรงค์ มูเซอ ผู้ต้องหาคดีค้าเฮโรอีน ซึ่งกรมสอบสวนคดีพิเศษ']

In [ ]:
model.predict('''segment sentence: ได้มีโอกาสออกมาเปิดใจเป็นครั้งแรก ในรายการ ตกมันส์บันเทิง ทางช่อง 9 MCOT HD สำหรับนักแสดงสาวระดับตัวแม่ของวงการบันเทิง บุ๋ม-ปนัดดา วงศ์ผู้ดี หลังจากที่ชื่อของเธอถูกโยงเข้ากับกรณีกระแสข่าวลือ ดาราสาวท้องแก่ใกล้คลอด ซึ่งกำลังเป็นที่จับตามองอยู่ ณ ขณะนี้
โดย บุ๋ม ปนัดดา ได้ชี้แจงรายละเอียดเกี่ยวกับกระแสฟีดแบคที่มาถึงตัวเธอ รวมทั้งเรื่องราวความสัมพันธ์กับสามีหุ่นล่ำ ก็อต-อธิป อนันทวรรณ ที่ได้เข้าพิธีวิวาห์อย่างเรียบง่ายไปเมื่อประมาณเกือบ 2 ปีก่อน...
ตั้งท้องจริงไหม ?
"ใช่ค่ะ แต่ที่ผ่านมาไม่มีใครทัก และตัวบุ๋มเองก็ไม่ได้ปิด อีกอย่างวัยนี้ท้องได้ก็บุญแล้ว เพราะอายุ 46 เป็นการท้องแบบธรรมชาติด้วย ซึ่งนั่นจึงเป็นอีกหนึ่งเหตุผลที่บุ๋มไม่ได้เล่าให้ใครฟัง เพราะบุ๋มอยากให้น้องแข็งแรงสมบูรณ์ที่สุดก่อน บุ๋มทราบว่าตัวเองตั้งท้องเมื่อประมาณช่วง 4 เดือนกว่าๆ น้องเป็นเด็กผู้ชาย กำหนดคลอดพรุ่งนี้ (22 พ.ย. 65)"
"เรื่องการแต่งงาน บุ๋มแต่งมาตั้งนานแล้วค่ะ 2 ปีแล้ว และเพิ่งมาท้อง ฝ่ายชายอยากมีลูก เขาอยากมีมานานแล้ว ไม่ใช่ความผิดพลาด ตอนที่รู้ว่าตัวเองตั้งท้อง มีคำถามในหัวเยอะมากว่าเราจะทำงานยังไง เนื่องจากเราเป็นคนบ้างาน และลูกจะแข็งแรงไหม บุ๋มถึงขั้นต้องส่งเลือดไปเช็กที่อเมริกา และพอรู้ว่าน้องแข็งแรงก็เหลือแค่ลุ้นว่าท้องจะใหญ่เมื่อไหร่ เพราะแม้กระทั่งช่วง 7-8 เดือนก็ยังไม่มีใครทัก จนกระทั่งถึงช่วงลอยกระทง สามีถามว่าคนต่อไปเมื่อไหร่ดี เขาอยากมีลูก แต่พอเขาเห็นอัลตร้าซาวด์ว่าเป็นลูกชายและหน้าเหมือนเขา เขาก็ยิ้มหน้าบานแล้ว"
"คลอดพรุ่งนี้เช้านะคะ ตื่นเต้นมาก คือบุ๋มท้องลูกคนนี้ห่างกับน้องอันดาตั้ง 16 ปี (ยิ้ม) ส่วนชื่อก็เดี๋ยวรอดูคุณพ่อเขาอีกทีว่าจะยังไง เพราะเขาค่อนข้างมีความเชื่อเรื่องนี้ และอาการแพ้ท้องต่างๆ ก็น้อยนะคะ ส่วนใหญ่เป็นแค่ง่วงนอน กับอยากกินข้าวไข่เจียว"''')

['ได้มีโอกาสออกมาเปิดใจเป็นครั้งแรก ในรายการ ตกมันส์บันเทิง ทางช่อง 9 MCOT HD สําหรับนักแสดงสาวระดับตัวแม่ของวงการบันเทิง บุ๋ม-ปนัดดา วงศ์ผู้ดี หลังจากที่ชื่อของเธอถูกโยงเข้ากับกรณีกระแสข่าวลือ ดาราสาวท้องแก่ใกล้คลอด ซึ่งกําลังเป็นที่จับตามองอยู่ ณ ขณะนี้ โดย บุ๋ม ปนัดดา ได้ชี้แจงรายละเอียดเกี่ยวกับกระแสฟีดแบคที่มาถึงตัวเธอ รวมทั้งเรื่องราวความสัมพันธ์กับสามีหุ่นล่ํา ก็อต-อธิป อนันทวรรณ ที่ได้เข้าพิธีวิวาห์อย่างเรียบง่ายไปเมื่อประมาณเกือบ 2 ปีก่อน...|ตั้งท้องจริงไหม?|"ใช่ค่ะ แต่ที่ผ่านมาไม่มีใครทัก และตัวบุ๋มเองก็ไม่ได้ปิด อีกอย่างวัยนี้ท้องได้ก็บุญแล้ว เพราะอายุ 46 เป็นการท้องแบบธรรมชาติด้วย ซึ่งนั่นจึงเป็นอีกหนึ่งเหตุผลที่บุ๋มไม่ได้เล่าให้ใครฟัง เพราะบุ๋มอยากให้น้องแข็งแรงสมบูรณ์ที่สุดก่อน บุ๋มทราบว่าตัวเองตั้งท้องเมื่อประมาณช่วง 4 เดือนกว่าๆ น้องเป็นเด็กผู้ชาย กําหนดคลอดพรุ่งนี้ (22 พ.ย. 65)"|"เรื่องการแต่งงาน บุ๋มแต่งมาตั้งนานแล้วค่ะ 2 ปีแล้ว และเพิ่งมาท้อง ฝ่ายชายอยากมีลูก เขาอยากมีมานานแล้ว|และเพิ่งมาท้อง']

In [ ]:
model.predict(''' 
ริชี สุนัค กลายเป็นนายกฯ อังกฤษ เชื้อสายเอเชียคนแรก
นายริชี สุนัค ได้รับเสียงสนับสนุนจาก ส.ส. พรรคคอนเซอร์เวทีฟท่วมท้น ให้รับตำแหน่งหัวหน้าพรรคและนายกรัฐมนตรีต่อจากนางลิซ ทรัสส์ หลังอยู่ในตำแหน่งเพียง 44 วัน ทำให้เขากลายเป็น นายกรัฐมนตรีเชื้อสายเอเชียคนแรกของสหราชอาณาจักร''')

['ริชี สุนัค กลายเป็นนายกฯ อังกฤษ เชื้อสายเอเชียคนแรก|นายริชี สุนัค ได้รับเสียงสนับสนุนจาก ส.ส. พรรคคอนเซอร์เวทีฟท่วมท้น ให้รับตําแหน่งหัวหน้าพรรคและนายกรัฐมนตรีต่อจากนางลิซ ทรัสส์ หลังอยู่ในตําแหน่งเพียง 44 วัน ทําให้เขากลายเป็น นายกรัฐมนตรีเชื้อสายเอเชียคนแรกของสหราชอาณาจักร|']

In [ ]:
model.predict("มึงเคยร้องไห้เพราะทำอะไรไม่เก่งซักอย่างป้ะ กัญชามีความจำเป็นอะไรต้องมาอยู่ในอาหารวะ โรงพยาบาลเป็นสถานที่หนึ่งที่คำว่า positive มีความหมายในเชิงไม่ดี")

['มึงเคยร้องไห้เพราะทําอะไรไม่เก่งซักอย่างป้ะ กัญชามีความจําเป็นอะไรต้องมาอยู่ในอาหารวะ|โรงพยาบาลเป็นสถานที่หนึ่งที่คําว่า positive มีความหมายในเชิงไม่ดี มีความหมายในเชิงไม่ดี']

In [ ]:
model.predict("ไอเดียนี้มาจากคณะอนุกรรมการอนุรักษ์พลังงาน (Energy Conservation Subcommittee) ซึ่งอยู่ภายใต้กระทรวงเศรษฐกิจ, การค้าและอุตสาหกรรมของญี่ปุ่น โดยจะมุ่งเน้นไปที่การควบคุมการใช้พลังงานของเครื่องใช้ไฟฟ้าประเภทเครื่องปรับอากาศของประชาชนในช่วงเวลาที่มีการใช้พลังงานไฟฟ้าสูงเกินกว่าที่ระบบจะสามารถจ่ายไหวซึ่งสุ่มเสี่ยงที่จะทำให้เกิดไฟดับเป็นวงกว้าง")[0].split('|')

['ไอเดียนี้มาจากคณะอนุกรรมการอนุรักษ์พลังงาน (Energy Conservation Subcommittee) ซึ่งอยู่ภายใต้กระทรวงเศรษฐกิจ, การค้าและอุตสาหกรรมของญี่ปุ่น โดยจะมุ่งเน้นไปที่การควบคุมการใช้พลังงานของเครื่องใช้ไฟฟ้าประเภทเครื่องปรับอากาศของประชาชนในช่วงเวลาที่มีการใช้พลังงานไฟฟ้าสูงเกินกว่าที่ระบบจะสามารถจ่ายไหวซึ่งสุ่มเสี่ยงที่จะทําให้เกิดไฟดับเป็นวงกว้าง',
 '']

In [ ]:
txt = '''องค์โชกุนทรงเป็นสตรีรูปร่างสูง มีพระเกศายาวสลวยสีม่วงถักเป็นเปียยาวมีปิ่นปักผมอยู่พระเศียรฝั่งขวา มีริบบิ้นสีแดงเข้มชิ้นเล็กเป็นสายรัดพระศอ ฉลองพระองค์เป็นชุดกิโมโนมีลวดลายหลากแบบเป็นทั้งสีม่วงและแดงเข้ม มีตราแผ่นดินอินาสึมะรัดอยู่กับผ้ารัดพระองค์โอบิในตำแหน่งใต้พระถันขวา ที่พระเพลามีถุงน่องสวมยาวขึ้นมาถึงพระอูรุตัดช่องรูปข้าวหลามตัดเล็กๆด้านบน ด้านหลังของฉลองพระองค์มีโบว์สีแดงเข้มพร้อมพู่ ฉลองพระบาทข้างขวามีดอกไม้สีม่วงอ่อนประดับอยู่สองดอก'''
model.predict(txt)[0].split('|')

['องค์โชกุนทรงเป็นสตรีรูปร่างสูง มีพระเกศายาวสลวยสีม่วงถักเป็นเปียยาวมีปิ่นปักผมอยู่พระเศียรฝั่งขวา มีริบบิ้นสีแดงเข้มชิ้นเล็กเป็นสายรัดพระศอ',
 'ฉลองพระองค์เป็นชุดกิโมโนมีลวดลายหลากแบบเป็นทั้งสีม่วงและแดงเข้ม มีตราแผ่นดินอินาสึมะรัดอยู่กับผ้ารัดพระองค์โอบิในตําแหน่งใต้พระถันขวา ที่พระเพลามีถุงน่องสวมยาวขึ้นมาถึงพระอูรุตัดช่องรูปข้าวหลามตัดเล็กๆด้านบน',
 'ด้านหลังของฉลองพระองค์มีโบว์สีแดงเข้มพร้อมพู่ ฉลองพระบาทข้างขวามีดอกไม้สีม่วงอ่อนประดับอยู่สองดอก']

# Evaluation

In [ ]:
# load trained T5 model
model.load_model("mt5", '/content/content/outputs/simplet5-epoch-0-train-loss-0.606-val-loss-0.1852', use_gpu=True)

In [ ]:
test_df

,source_text,target_text
0,ม็อบสมาพันธ์ชุมชนบุกทำเนียบทวงคำตอบนายกฯ เมื่อ...,ม็อบสมาพันธ์ชุมชนบุกทำเนียบทวงคำตอบนายกฯ|เมื่อ...
1,ความเป็นจริง และแก้ปัญหาเรื่องบัตรเครดิต รถยนต...,ความเป็นจริง และแก้ปัญหาเรื่องบัตรเครดิต รถยนต...
2,สามารถหาคนค้ำประกันได้ จึงต้องให้พิจารณาประเด็...,สามารถหาคนค้ำประกันได้ จึงต้องให้พิจารณาประเด็...
3,ตำรวจกระจายกำลังตามจุดต่างๆ โดยรอบศาลฎีกา ก่อน...,ตำรวจกระจายกำลังตามจุดต่างๆ โดยรอบศาลฎีกา ก่อน...
4,วิเชียรพจน์โพธิ์ศรี รักษาราชการแทนผู้บัญชาการต...,วิเชียรพจน์โพธิ์ศรี รักษาราชการแทนผู้บัญชาการต...
...,...,...
1650,กีฬาในการแข่งขันกีฬาระดับนานาชาติ 15 รายการในร...,กีฬาในการแข่งขันกีฬาระดับนานาชาติ 15 รายการในร...
1651,เพื่อใช้เป็นชุดนักกีฬาทีมชาติไทยในการเข้าร่วมก...,เพื่อใช้เป็นชุดนักกีฬาทีมชาติไทยในการเข้าร่วมก...
1652,"ครั้งที่ 1, เอเชียนเกมส์ครั้งที่ 16, เอเชียนบี...","ครั้งที่ 1, เอเชียนเกมส์ครั้งที่ 16, เอเชียนบี..."
1653,"""เฮียฮ้อ""ลุย ปรับอาวอง เพราะเกิดวิกฤติเศรษฐกิจ...","""เฮียฮ้อ""ลุย ปรับอาวอง|เพราะเกิดวิกฤติเศรษฐกิจ..."


In [ ]:
predicted = ''
for i in test_df['source_text']:
  result = model.predict(i)[0].replace( 'ํา',"ำ")
  predicted += result

In [ ]:
predicted

'วันที่ 17 ส.ค.) นายจำรัส อินทุมาร ประธานสมาพันธ์ชุมชนแห่งประเทศไทยพร้อมตัวแทนประชาชนประมาณ 50 คน ได้เดินทางมมาที่ทำเนียบรัฐบาล เพื่อติดตามเร่งรัดความคืบหน้าที่ได้ร้องเรียนให้รัฐบาลช่วยเหลือทุนประกอบอาชีพให้ประชาชน ตามโครงการสร้างคน สร้างงาน สร้างรายได้ ตามหลักปญาเศรษฐกิจพอเพียง กลุ่มละ 1 ล้านบาท รวมทั้งให้ยุติการบังคับคดีเกี่ยวกับที่อยู่อาศัย โดยให้ธนาคารของรัฐไถ่ถอนหรือซื้อคืนบ้าน ที่ทำกินให้กับประชาชน และผ่อนส่งราคาที่เป็นธรรมกับธนาคารของรัฐ และอนุญาตให้กู้เงินใช้หนี้นอกระบบให้กับประชาชนตามที่มีหนี้สินตามขอให้นายอภิสิทธิ์ เวชชาชีวะ นายกรัฐมนตรี มีบัญชาไปยังนายกอร์ปศักดิ์ สภาวสุ รองนายกรัฐมนตรี นายกรณ์ จาติกวณิช รมว.คลัง ดำเนินการช่วยเหลือประชาชนผู้เดือดร้อน|อีกทั้งนายกรณ์ เคยระบุว่า จะให้มีการตั้งธนาคารชุมชนเพื่อช่วยเหลือประชาชนที่ได้รับความเดือดร้อน ซึ่งตนเห็นด้วยที่จะมีการดำเนินการในโครงการนี้ แต่อยากให้รัฐบาลชี้แจงเงื่อนไข รายละเอียดต่าง ๆ ของการดำเนินงานธนาคารชุมชน โดยเฉพาะประเด็นที่ให้ข้าราชการค้ำประกันให้กับชาวบ้านรัฐจึงจะปล่อยกู้ เนื่องจากชาวบ้านบางคนมีฐานะยากจนและไม่หากรัฐบา

In [ ]:
gold = ''
for i in test_df['target_text']:
  gold += i

In [ ]:
gold

'ม็อบสมาพันธ์ชุมชนบุกทำเนียบทวงคำตอบนายกฯ|เมื่อเวลา 11.00 น. วันนี้(17 ส.ค.) นายจำรัส อินทุมาร ประธานสมาพันธ์ชุมชนแห่งประเทศไทยพร้อมตัวแทนประชาชนประมาณ 50 คน ได้เดินทางมมาที่ทำเนียบรัฐบาล เพื่อติดตามเร่งรัดความคืบหน้าที่ได้ร้องเรียนให้รัฐบาลช่วยเหลือทุนประกอบอาชีพให้ประชาชน|ตามโครงการสร้างคน สร้างงาน สร้างรายได้ ตามหลักปรัชญาเศรษฐกิจพอเพียง กลุ่มละ 1 ล้านบาท รวมทั้งให้ยุติการบังคับคดีเกี่ยวกับที่อยู่อาศัย|โดยให้ธนาคารของรัฐไถ่ถอนหรือซื้อคืนบ้าน ที่ทำกินให้กับประชาชน และผ่อนส่งราคาที่เป็นธรรมกับธนาคารของรัฐ และอนุญาตให้กู้เงินใช้หนี้นอกระบบให้กับประชาชนตามที่มีหนี้สินตามความเป็นจริง และแก้ปัญหาเรื่องบัตรเครดิต รถยนต์ รถมอเตอร์ไซค์ ที่ใช้ประกอบอาชีพจะโดนยึด เพื่อจะได้ใช้ประกอบอาชีพต่อไปได้|นายจำรัส กล่าวว่า ขอให้นายอภิสิทธิ์ เวชชาชีวะ นายกรัฐมนตรี มีบัญชาไปยังนายกอร์ปศักดิ์ สภาวสุ รองนายกรัฐมนตรี นายกรณ์ จาติกวณิช รมว.คลัง ดำเนินการช่วยเหลือประชาชนผู้เดือดร้อน|อีกทั้งนายกรณ์ เคยระบุว่า จะให้มีการตั้งธนาคารชุมชนเพื่อช่วยเหลือประชาชนที่ได้รับความเดือดร้อน ซึ่งตนเห็นด้วยที่จะมีการดำเนินการใ

In [ ]:
[x for x in predicted.split('|') if x in gold.split('|')]

['ภูมิใจไทยป่วน"อนุทิน"งัดข้อ"เนวิน" เหตุไม่ลงตัวเก้าอี้ปลัดมหาดไทย หลัง"ห้อย" จะดันอธิบดีท้องถิ่น แต่ "ลูกมท.1" ไม่ยอม "ชวรัตน์" ชี้สเปคต้องเข้ากับนักการเมืองได้',
 'ศาลฎีกาแผนกคดีอาญาของผู้ดำรงตำแหน่งทางการเมือง มีคำสั่งให้เลื่อนการอ่านคำพิพากษาคดีทุจริตโครงการจัดซื้อกล้ายางพารา และมีคำสั่งออกหมายจับอดีตรัฐมนตรีว่าการกระทรวงพาณิชย์ ที่ไม่เดินทางมาศาลฯ',
 'แกนนำพรรคภูมิใจไทย "จาตุรนต์ ชิดชอบ" ปฏิเสธข่าวบินหารือผู้บัญชาการเหล่าทัพที่ประเทศสิงคโปร์ พร้อมยันไปฟังคำพิพากษาคดีกล้ายาง มั่นใจบริสุทธิ์ ไม่คิดฎีกาอภัยโทษ',
 'รมช.มหาดไทย"บุญจง" ปัดขนคนเข้ากรุง เพื่อก่อกวนเสื้อแดง พร้อมเชื่อ จะไม่มีเหตุปะทะกัน',
 'ชะลอโครงการชุมชนพอเพียง รอผลสอบกรรมการจับผิดโกง',
 'รัฐบาลชะลอโครงการชุมชนพอเพียงทั่วประเทศ เผยต้องการสางปัญหาทุจริตก่อน ขณะที่ผู้บริหารยอมรับที่ผ่านมาแผนประชาสัมพันธ์ไม่บรรลุเป้า เนื่องจากติดระบบราชการ ส่งผลชาวบ้านรับข้อมูลไม่ครบ',
 'ด้าน "กอร์ปศักดิ์" วาง 5 มาตรการสอบโครงการ ส่วน "นายนิพิฏฐ์ อินทรสมบัติ" เรียก 3 ส.ข.สอบอีกรอบ',
 'ประธานวิปรัฐบาล ระบุ อดีตนายกรัฐมนตรีไม่มีความจริงใจใน

In [ ]:
len([x for x in predicted.split('|') if x in gold.split('|')])

186

In [ ]:
len(predicted.split('|'))

1273

In [ ]:
len(gold.split('|'))

5251

In [ ]:
# Precision
# จำนวนที่ทำนายถูก / จำนวนที่ทำนายมาทั้งหมด
precision = len([x for x in predicted.split('|') if x in gold.split('|')]) / len(predicted.split('|'))
precision

0.14611154752553024

In [ ]:
# Recall
# จำนวนที่ทำนายถูก / จำนวนใน gold standard
recall = len([x for x in predicted.split('|') if x in gold.split('|')]) / len(gold.split('|'))
recall

0.03542182441439726

In [ ]:
# F1 = 2PR/(P+F)
f1_score = 2 * precision * recall / (precision + recall)
f1_score

0.05702023298589822